<br/>

<img src="https://uploads-ssl.webflow.com/614b1fe22fa8b90ef41aeffe/6265cb48f9496b1cefc9ab75_logotipo-mbit-39.png" width="200px" align="right" CLASS="TextWrap" style="background-color:#2a3f3f;">

<font color="#2a3f3f" size=5>Proyecto de consolidación - M02</font>

<font color="#2a3f3f" size=4>Creación de una aplicación apificada basada en microservicios</font>

<br/>
<div style="text-align: right">
<font color="#2a3f3f" size=3>Javier Cózar - javier.cozar@mbitschool.com</font><br>
<font color="#2a3f3f" size=3>Máster en Data Engineering </font><br>
</div>


# Introducción

En este proyecto vamos a crear una aplicación basada en microservicios, utilizando la tecnología Docker para ejecutar cada microservicio en un entorno reproducible.

La aplicación permitirá subir fotos, las cuales etiquetará automáticamente, y las almacenará en una carpeta además de crear una base de datos donde existirá toda esta información (paths a imágenes y etiquetas  asignadas). Esta información será utilizada para permitir también buscar imágenes por una etiqueta concreta.

Existirán los siguientes microservicios:

- API: implementada en Flask y servida mediante waitress.
- Base de datos: utilizaremos una base de datos MySQL 8.0.

A continuación se describen las propiedades de cada elemento y las tareas a realizar:

## Descripción


### Base de datos

Se utilizará una base de datos MySQL versión 8.0. Explorar la documentación de la imágen en [dockerhub](https://hub.docker.com/_/mysql). Debemos:

- Usar una base de datos llamada `Pictures`
- Usar un usuario con nombre `mbit` y contraseña `mbit`.

El modelo de datos que se debe usar es el siguiente:

#### Tabla `pictures`

Tabla que contendrá una fila por cada imagen almacenada en el sistema. Tiene las siguientes columnas:

- `id`: Columna de enteros que se corresponden con un ID autoincrementado para cada imagen. Es la **Primary Key**.
- `path`: String que identifica el path absoluto donde está almacenada la imagen.
- `date`: String que identifica la fecha en la que se creó la imagen, en formato `YYYY-MM-DD HH:MM:SS`.

#### Tabla `tags`

Tabla que contendrá las tags asociadas a cada imagen. Tiene las siguientes columnas:

- `tag`: Nombre de la tag asociada a la imagen (como mucho 32 caracteres).
- `picture_id`: ID de la imagen que está asociada a la tag. Es una **Foreign Key**.
- `confidence`: Nivel de confianza de la tag asociada a la imagen.
- `date`: String que identifica la fecha en la que se creó la imagen, en formato `YYYY-MM-DD HH:MM:SS`.

La **Primary Key** será compuesta por las columnas `tag` y `picture_id`.

### API

Se implementará una API en Flask. Se utilizará la imágen de [dockerhub](https://hub.docker.com/_/python) con la tag `3.11`. Será necesario construir nuestro propio `Dockerfile` para crear la imagen que contenga nuestro código custom que, además de implementar la funcionalidad necesaria con Flask, sirva la API utilizando `waitress` en el puerto 80 del contenedor.

La API debe tener tres endpoints:

#### POST image

Este endpoint espera como input un body que será un `json` con un campo `data` que es la imagen codificada en base64. También se especificará un query parameter **opcional** llamado `min_confidence`, que nos servirá para exigir ese valor de certeza a las etiquetas generadas para la imagen. Su valor por defecto es `80`. Una vez recibida utilizará un [servicio cloud mediante una API](https://imagga.com/) para extraer tags a partir de esta imagen. Esta API requiere que le pasemos la imagen como una URL pública, por lo que usaremos en primer lugar otro [servicio cloud mediante una API](https://docs.imagekit.io/) para subir temporalmente esta imagen a la nube. Por lo tanto, la secuencia de pasos será:

1. Usamos [imagekit.io](https://docs.imagekit.io/api-reference/upload-file-api/server-side-file-upload#uploading-base64-encoded-file-with-some-tags) para subir la imagen a la nube de forma pública.
2. Usamos https://imagga.com/ para extraer tags a partir de la imagen subida anteriormente.
3. Borramos la imagen subida a https://docs.imagekit.io/ usando la api [delete](https://docs.imagekit.io/api-reference/media-api/delete-file).
4. Almacenamos en una carpeta determinada la imagen (cuidado de no machacar imágenes ya existentes, puedes generar nombres aleatorios para las imágenes). Debemos usar un volumen de docker en el path de esta carpeta.
5. Almacenamos información en la base de datos:
     - En la tabla `pictures` almacenaremos el path de la imagen que tendrá un id asociado. También almacenaremos la fecha en la que se ha introducido como un string en formato `YYYY-MM-DD HH:MM:SS`.
     - En la tabla `tags` almacenaremos una fila por cada tag asociado a la imagen, donde tendremos los campos `tag`, `picture_id` (foreign key) y la fecha en la que se ha introducido la imagen como un string en formato `YYYY-MM-DD HH:MM:SS`. También se almacenará qué `confidence` tiene esta tag para la imagen.


La **respuesta** de este endpoint debe ser un **json** con los siguientes campos:

**NOTA**: será necesario en primer lugar crearse una cuenta en `https://docs.imagekit.io/` y en `https://imagga.com/`, y utilizar credenciales correctamente:

- `imagekit.io`: ir a dashboard, crearnos una cuenta, y al loguearnos ir a **Developer options**. Ahí veremos nuestro `URL-endpoint`, y unas credenciales por defecto ya creadas (`Public Key` y `Private Key`). Aunque se puede utilizar `requests`, vamos a usar la librería para Python que nos proporciona la plataforma que simplifica el proceso de autenticación. Para ello debemos instalar la librería `imagekitio`. Podemos usar el siguiente código para subir y borrar imágenes:

```python
from imagekitio import ImageKit

imagekit = ImageKit(
    public_key='your public_key',
    private_key='your private_key',
    url_endpoint = 'your url_endpoint'
)

with open("image.jpg", mode="rb") as img:
    imgstr = base64.b64encode(img.read())

# upload an image
upload_info = imagekit.upload(file=imgstr, file_name="my_file_name.jpg")
# la url es accesible mediante `upload_info.url`

# delete an image
delete = imagekit.delete_file(file_id=upload_info.file_id)
```
- `imagga.com`: Crearnos una cuenta y al loguearnos ir a Dashboard. Ahí veremos nuestra `API Key`, `API Secret`. Podemos usar el siguiente código para obtener las etiquetas para una imagen:

```python
api_key = 'your api key'
api_secret = 'your api secret'
image_url = 'the image url'

response = requests.get(f"https://api.imagga.com/v2/tags?image_url={image_url}", auth=(api_key, api_secret))

tags = [
    t["tag"]["en"]
    for t in response.json()["result"]["tags"]
    if t["confidence"] > min_confidence
]
```

#### GET images

Este endpoint sirve para obtener una lista de imágenes que cumplan un filtro. Se proporcionará mediante query parameters la siguiente información:

- `min_date`/`max_date`: opcionalmente se puede indicar una fecha mínima y máxima, en formato `YYYY-MM-DD HH:MM:SS`, para obtener imágenes cuya fecha de registro esté entre ambos valores. Si no se proporciona `min_date` no se filtrará ningúna fecha inferiormente. Si no se proporciona `max_date` no se filtrará ningúna fecha superiormente.

- `tags`: optionalmente se puede indicar una lista de tags. Las imágenes devueltas serán aquellas que incluyan **todas** las tags indicadas. El formato de este campo será un string donde las tags estarán separadas por comas, por ejemplo `"tag1,tag2,tag3"`.

El endpoint devolvera una respuesta que será una lista de objetos imagen con los siguientes campos:

- `id`: identificador de la imagen
- `size`: tamaño de la imagen en KB
- `date`: fecha en la que se registró la imagen, en formato `YYYY-MM-DD HH:MM:SS`
- `tags`: lista de nombres de tags asociados a la imágen

#### GET image

Este endpoint sirve para descargarse una imágen y sus propiedades. Se proporcionará mediante path parameter el id de la imagen y se devolverá un json con los siguientes campos:

- `id`: identificador de la imagen
- `size`: tamaño de la imagen en KB
- `date`: fecha en la que se registró la imagen, en formato `YYYY-MM-DD HH:MM:SS`
- `tags`: lista de objetos identificando las tags asociadas a la imágen. Cada objeto tendrá el siguiente formato:
    - `tag`: nombre de la tag
    - `confidence`: confianza con la que la etiqueta está asociada a la imagen
- `data`: imagen como string codificado en base64


#### GET tags

Este endpoint sirve para obtener una lista de tags (teniendo en cuenta cualquier tag asignada a una imagen registrada) que cumplan un filtro. Se proporcionará mediante query parameters la siguiente información:

- `min_date`/`max_date`: opcionalmente se puede indicar una fecha mínima y máxima, en formato `YYYY-MM-DD HH:MM:SS`, para obtener imágenes cuya fecha de registro esté entre ambos valores. Si no se proporciona `min_date` no se filtrará ningúna fecha inferiormente. Si no se proporciona `max_date` no se filtrará ningúna fecha superiormente.

El endpoint devolvera una respuesta que será una lista de objetos imagen con los siguientes campos:

- `tag`: nombre de la etiqueta
- `n_images`: número de imágenes que tienen asociada esta tag
- `min_confidence`, `max_confidence`, `mean_confidence`: confianza mínima, máxima y media de esta tag para todas las imágenes con las que está asignada
las dos APIs utilizadas. 

## Tareas

A continuación se **resume** el conjunto de tareas a realizar, así como el formato del entregable a subir al assigment en Microsoft Teams.

1. Crear una estructura de proyecto en Flask para producción (carpeta que contenga un fichero `__init__.py`, `models.py`, `controller.py` y `views.py`).
1. Implementar la API descrita en el enunciado anterior.
1. Crear un fichero `Dockerfile` que parta de `python:3.11`, incluya el código de la API, instale las dependencias de Python necesarias, e inicie el servidor WSGI usando `waitress`.
1. Crear un fichero `docker-compose.yml` que describa el microservicio de la base de datos MySQL:8.0 y la API en Flask. También debe especificar un volumen de Docker y montarlo en la carpeta del microservicio de la API para que las imágenes se guarden en el volumen. Por último, debe exponer el puerto 80 del contenedor de la API en el puerto 80 del host. Si te da algún problema, exponlo en el puero 8080 del host en su lugar. Para especificar las credenciales de la base de datos utilizaremos variables de entorno definidas en el propio docker-compose.yml.
1. Para especificar las credenciales para utilizar los servicios  `imagekit` y `imagga` se creará un fichero llamado `credentials.json` que la incluirá. Este fichero se proporcionará a la imagen de docker de la API para que lo cargue y las utilice.
1. Iniciar la aplicación usando docker compose. Es **importante** que antes de probarlo te conectes al contenedor de Docker y crees las tablas necesarias.
1. Testear la API haciendo uso de la librería `requests`. Usa esta misma libreta para hacer esas llamadas.
1. Crear un repositorio **público** en **GitHub** y subir el proyecto. Es **importante** que no subas el fichero `credentials.json` con las credencialesde las dos APIs utilizadas. **Opcionalmente** puedes utilizar un fichero [.gitignore](https://git-scm.com/docs/gitignore) para ello.


### Entregable

Se entregará esta misma libreta completando la siguiente información en las siguientes celdas:

- **URL del repositorio público de GitHub** donde has subido el código desarrollado. Es importante que verifiques que en el repositorio se encuentra toda la información, que no se incluye información sensible (fichero `credentials.json` con las credencialesde las dos APIs utilizadas), y que el respositorio es accesible de forma pública (puedes probar a acceder a la URL usando el navegador en modo incógnito).
- **Código SQL utilizado para crear las tablas** `pictures` y `tags`.
- **Uso de `requests` invocando la API**. Ejecuta las celdas y no borres los outputs de las celdas en la libreta que adjuntes como entregable.
- Cualquier **comentario** que consideres importante.

In [1]:
# Completar
https://github.com/casraysa/MDE/
# Incluye la URL al repositorio público de GitHub

In [ ]:
# Completar
# Incluye el código SQL para crear las tablas pictures y tags
CREATE TABLE pictures (
    id int NOT NULL AUTO_INCREMENT,
    path TEXT NOT NULL,
    date varchar(25) NOT NULL,	
    PRIMARY KEY (id)
);


CREATE TABLE tags (
    tag varchar(32) NOT NULL,
    picture_id int NOT NULL,
    confidence float,
    date varchar(25) NOT NULL,
    PRIMARY KEY (tag, picture_id),
    FOREIGN KEY (picture_id) REFERENCES pictures(id)
);

# Otro comandos necesarios para configurar la base de datos
mysql -u root -p
CREATE DATABASE Pictures;
GRANT ALL PRIVILEGES ON Pictures.* TO 'mbit'@'%';
FLUSH PRIVILEGES;

In [4]:
# Completar

# Utiliza la librería requests para probar la aplicación implementada

In [ ]:
# Completar (opcional)

# Incluye en una celda de tipo markdown cualquier comentario que consideres relevante